In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_train=pd.read_csv("/kaggle/input/cassava-leaf-disease-classification/train.csv")

In [ ]:
df_train_text=df_train.copy()

In [ ]:
os.mkdir("/kaggle/working/train_img")


In [ ]:
labels=[0,1,2,3,4]
for i in range(len(labels)):
    k=str(i)
    os.mkdir("/kaggle/working/train_img/"+k)

In [ ]:
import shutil
from tqdm import tqdm

for j in tqdm(range(len(df_train["image_id"]))):
    img=df_train["image_id"].iloc[j]
    if(df_train["label"].iloc[j]==0):
        shutil.copy("/kaggle/input/cassava-leaf-disease-classification/train_images/"+img,"/kaggle/working/train_img/0/")
    elif(df_train["label"].iloc[j]==1):
        shutil.copy("/kaggle/input/cassava-leaf-disease-classification/train_images/"+img,"/kaggle/working/train_img/1/")
    elif(df_train["label"].iloc[j]==2):
        shutil.copy("/kaggle/input/cassava-leaf-disease-classification/train_images/"+img,"/kaggle/working/train_img/2/")
    elif(df_train["label"].iloc[j]==3):
        shutil.copy("/kaggle/input/cassava-leaf-disease-classification/train_images/"+img,"/kaggle/working/train_img/3/")
    elif(df_train["label"].iloc[j]==4):
        shutil.copy("/kaggle/input/cassava-leaf-disease-classification/train_images/"+img,"/kaggle/working/train_img/4/")
print("Copying Files Done!")


In [ ]:
numeric_to_class_name = {0:"Cassava Bacterial Blight (CBB)",
                         1:"Cassava Brown Streak Disease (CBSD)",
                         2:"Cassava Green Mottle (CGM)",
                         3:"Cassava Mosaic Disease (CMD)",
                         4:"Healthy"}

df_train_text['label'].replace(numeric_to_class_name, inplace=True)

In [ ]:
df_train_text.label.unique()

### There are 5 classes for prediction in this dataset :

* Healthy -> The Leaf is healthy
* Cassava Bacterial Blight (CBB)
* Cassava Brown Streak Disease (CBSD)
* Cassava Green Mottle (CGM)
* Cassava Mosaic Disease (CMD)

In [ ]:
import seaborn as sns
sns.set(rc={'figure.figsize':(24,12)})
sns.countplot(data=df_train_text,x="label")


In [ ]:
label_count=df_train_text[["label"]].value_counts().reset_index()
label_count.columns=["Label","Number of Observations"]

In [ ]:
label_count.head()

In [ ]:
import plotly.express as px
fig=px.pie(label_count,values="Number of Observations",names="Label")
fig.show()

In [ ]:
import os
from PIL import Image
import random 
import matplotlib.pyplot as plt



In [ ]:
def show_images(label_name):
    plt.figure(figsize=((10,8)))
    for i in range(0,6):
        plt.subplot(3,2,i+1)
        path="/kaggle/working/train_img/"+str(label_name)+"/"
        files=os.listdir(path)
        k=random.choice(files)
        img=plt.imread(path+k)
        plt.imshow(img)
        plt.axis("off")
    plt.tight_layout()
        
            
    

## Examples of leaves having disease Cassava Bacterial Blight (CBB)

In [ ]:
show_images(0)

## Examples of leaves having disease Cassava Brown Streak Disease (CBSD)

In [ ]:
show_images(1)

## Examples of leaves having disease Cassava Brown Streak Disease (CBSD)

In [ ]:
show_images(2)

## Examples of leaves having disease Cassava Mosaic Disease (CMD)

In [ ]:
show_images(3)

## Examples of leaves which are Healthy

In [ ]:
show_images(4)

## Model Building and Training

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
img_0.size

In [ ]:
ImgDataGen=ImageDataGenerator(
    rotation_range=0.5, width_shift_range=0.3,
    height_shift_range=0.3,horizontal_flip=True, vertical_flip=True,
    validation_split=0.2
)
DataGenDir=ImgDataGen.flow_from_directory("/kaggle/working/train_img/",
                                         target_size=(224,224),
                                         batch_size=64,
                                         class_mode="categorical")

In [ ]:
!pip install -q efficientnet

In [ ]:
import efficientnet.tfkeras as efn

In [ ]:
# instantiating the model in the startegy scope creating the model on the TPU
model=tf.keras.Sequential([efn.EfficientNetB0(input_shape=(224,224,3),
                                             weights="imagenet",
                                             include_top=False),
                           tf.keras.layers.Flatten(),
                           tf.keras.layers.Dropout(0.2),
                           tf.keras.layers.Dense(5,activation="softmax")
                                             
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),loss="categorical_crossentropy",metrics=["acc"])

In [ ]:
model.summary()

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


In [ ]:
if tf.test.gpu_device_name(): 

    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

else:

   print("Please install GPU version of TF")

### To be continued.....